In [2]:
#define function that scrapes one page at a time and returns a panda dataframe
def scrape_page(page_num):
    import requests  # b_soup_1.py
    import re
    from bs4 import BeautifulSoup
    #spoof the user agent to make page scrapable
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
#define position search term (one word) and location
    position='analyst'
    location='Pittsburgh,+PA'
    page_str=str(page_num)
#url for search query
    url='https://www.indeed.com/jobs?q='+position+'&l='+location+'&start='+page_str
    search_page=requests.get(url,headers=headers)
    search_page
    soup=BeautifulSoup(search_page.content,'html.parser')
    all_cards=soup.find_all(class_='jobsearch-SerpJobCard row result')
#empty list to store job posting urls
    job_urls=[]
#it seems to be easier to pull posting dates and locations from search page, so do that here
    posting_dates=[]
    locations=[]
#iterate through all_cards, extract job posting urls, posting dates, and locations
    for card in all_cards:
    #urls
        job_id=card['data-jk']
        job_url='https://www.indeed.com/viewjob?jk='+job_id
        job_urls.append(job_url)
    #posting dates
    #some jobs don't have posting dates so try them
        try:
            job_post_date=card.find(class_='date').get_text()
        except:
            job_post_date=' '
        posting_dates.append(job_post_date)
    #locations
        job_location=card.find(class_='location').get_text()
        locations.append(job_location)
#locations
#lets make this dataframe
#declare the empty lists that will become columns
    position_name=[]
    employer=[]
    full_description=[]
#in the future, we can develop a list of keywords that we'll 
  #search postings for, but for now, just return all capitalized
   #words besides common words
    capital_words=[]
    for job_url in job_urls:
    #retrieve job page
        job_page=requests.get(job_url,headers=headers)
    #make job page into BeautifulSoup object
        job_soup=BeautifulSoup(job_page.content,'html.parser')
    #retrieve the job title
        job_position=job_soup.find(class_='icl-u-xs-mb--xs').get_text()
    #retrieve company name from job rating line on page
        job_rating_line=job_soup.find(class_='jobsearch-InlineCompanyRating')
        job_employer=job_rating_line.find(class_='icl-u-lg-mr--sm').get_text()
    #retrieve full job description
        job_descrip=job_soup.find(class_='jobsearch-JobComponent-description').get_text()
    #replace line breaks in description with spaces
        job_descrip.replace('\n',' ')
    #find capitalized words in job description
        capitals=re.findall('([A-Z][a-z]+)', job_descrip)
    #replace commonly capitalized words
        for x in ['The','Our','A','As','On','Using','We', 'In', 'Some']:
            try:
                capitals[:]=(value for value in capitals if value!=x)
            except:
                capitals
    #make capitals into a set instead of a list
        capitals=set(capitals)
        capital_string=', '.join(capitals)
    #append all of the items created to the empty lists from last step
        position_name.append(job_position)
        employer.append(job_employer)
        full_description.append(job_descrip)
        capital_words.append(capitals)
        
#store results in dictionary, then data frame
    import pandas as pd
    result_dict={'Job Title':position_name,
                'Employer':employer,
                'Location':locations,
                'Posting Date':posting_dates, 
                'Posting Url':job_urls,
                'Full Job Description':full_description,
                'Keywords':capital_words
                }
    result_frame=pd.DataFrame(result_dict)
    return(result_frame)

In [6]:
import pandas as pd
df=scrape_page(0)
for x in range(1,10):
    temp=scrape_page(x)
    df=pd.concat([df,temp])


In [8]:
df

,Job Title,Employer,Location,Posting Date,Posting Url,Full Job Description,Keywords
0,Business Analyst,"Liberty Tire Recycling, LLC","Pittsburgh, PA 15212",2 days ago,https://www.indeed.com/viewjob?jk=2b6b7499b165...,"$50,000 - $55,000 a yearCompany OverviewLibert...","{Assist, Recycling, North, Board, Bachelor, At..."
1,IT Business Analyst,Pepon inc,"Pittsburgh, PA",5 hours ago,https://www.indeed.com/viewjob?jk=e3e1de7c3972...,ContractExperience: 10+ Years business analyst...,"{Exceptional, Location, Microsoft, No, Contrac..."
2,Business Process Analyst,Polyconcept North America,"New Kensington, PA 15068",,https://www.indeed.com/viewjob?jk=42ad913a9fce...,Customer Operations Business Process Analyst\n...,"{Perform, Supports, Translate, Cases, Microsof..."
3,Eligibility Analyst,Zenith American,"Pittsburgh, PA 15222 (Strip District area)",1 day ago,https://www.indeed.com/viewjob?jk=d7653382d3e3...,Title: Eligibility Analyst Department: Contrib...,"{Accounting, Key, Type, National, Analyst, Acc..."
4,Business Analyst,"Liberty Tire Recycling, LLC","Pittsburgh, PA 15212",2 days ago,https://www.indeed.com/viewjob?jk=2b6b7499b165...,"$50,000 - $55,000 a yearCompany OverviewLibert...","{Assist, Recycling, North, Board, Bachelor, At..."
5,Business Analyst,Concept Technology LLC,"Pittsburgh, PA",11 days ago,https://www.indeed.com/viewjob?jk=b29010d4ac11...,"$50,000 - $60,000 a yearHello Everyone,Our rap...","{Software, Thanks, Hello, Everyone, Type, Data..."
6,Help Desk Analyst,The University of Pittsburgh,"Pittsburgh, PA",1 day ago,https://www.indeed.com/viewjob?jk=e6ffce7a8132...,"$26,988 - $43,212 a yearHelp Desk Analyst\nCom...","{Systems, Level, Pennsylvania, Cover, Relocati..."
7,Data Analyst,The University of Pittsburgh,"Pittsburgh, PA",14 days ago,https://www.indeed.com/viewjob?jk=21100e0462be...,"$35,100 - $56,784 a yearData Analyst\nSchool o...","{Systems, Level, Pennsylvania, Cover, Relocati..."
8,Entry Level Data Analyst,"IQlogg, Inc","Pittsburgh, PA",15 days ago,https://www.indeed.com/viewjob?jk=111e2191b9de...,Qualifiers: Bachelor’s degree in Computer Scie...,"{Intermediate, Tableau, Qualifiers, Bachelor, ..."
9,Open Source Intelligence Investigative Analyst,Social Media Information,"Pittsburgh, PA 15212",30+ days ago,https://www.indeed.com/viewjob?jk=dbdb56032739...,"$35,000 a yearJob Description\n\nSMI is hiring...","{Description, Microsoft, Relocation, Free, Typ..."


In [ ]:
df.toexce